In [495]:
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

import pandas as pd
from datetime import datetime
import re



In [496]:
connection = DatabaseConnector()
engine = connection.init_db_engine()

extractor = DataExtractor()
df = extractor.read_rds_table(engine, "legacy_users")

In [497]:
# class DataCleaning():
  
#   def clean_user_data(self, pandas_dataframe):
#     df = pandas_dataframe

#     # df.drop("first_name", axis=1, inplace=True)

#     return cleaned df




Checking for missing/null values in the columns

In [498]:

df.isnull().sum()

index            0
first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64

The above shows that there are no missing or null values in any column.

We can check now that the index is unique.

In [499]:
is_unique = df["index"].is_unique
is_unique

True

In [500]:
is_unique = df["user_uuid"].is_unique
is_unique

False

In [501]:
duplicate_uuids = df[df.duplicated("user_uuid", keep=False)]
duplicate_uuids

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
866,867,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1022,1023,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1805,1807,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2103,2103,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2437,2439,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2739,6526,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2764,2764,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4984,4987,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
5307,5310,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
6920,6927,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


`pd.Series(range(len(df)))` creates a Series (like a column) of numbers from 0 to length of the dataframe -1. The `.equals()`function compares to see if two objects are the same. Essentially, we're comparing the index column to the range. Returns True if equal.

In [502]:
is_sequential = df['index'].equals(pd.Series(range(len(df))))
is_sequential

False

Given that it's not sequential, but there are `(len(df))` unique, non-null values, it's possible that the index column is just messed up. We can sort the dataframe by the index column, and compare it to the Series again.

In [503]:
sorted_df = df.sort_values(by="index")
sorted_df.reset_index(drop=True, inplace=True)
is_sequential = sorted_df["index"].equals(pd.Series(range(len(df))))
# sorted_df.tail()
is_sequential

True

In [504]:
sorted_df.head()

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


We can drop the index column, as it's not adding any useful information

In [505]:
sorted_df = sorted_df.drop(columns="index")
sorted_df.head()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [506]:
sorted_df.dtypes

first_name       object
last_name        object
date_of_birth    object
company          object
email_address    object
address          object
country          object
country_code     object
phone_number     object
join_date        object
user_uuid        object
dtype: object

In [507]:
null_search_id = sorted_df.loc[sorted_df["user_uuid"] == "NULL"]
null_search_id

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
867,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1023,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1807,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2103,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2439,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2764,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4987,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
5310,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
6526,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
6927,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


Let's drop all of the values that match "NULL". We're dropping the rows that match our null_search_id by passing it to the drop function.

In [508]:
filtered_df = sorted_df.drop(null_search_id.index)
filtered_df = filtered_df.reset_index(drop=True)

Test again for uniqueness in the filtered dataframe

In [509]:
unique_test = filtered_df["user_uuid"].is_unique
unique_test

True

Let's look at the date of birth column

In [510]:
unique_dob_values = filtered_df["date_of_birth"].nunique()
unique_dob_values

11359

In [511]:
date_of_birth_series = filtered_df["date_of_birth"]
len(date_of_birth_series)

15299

In [512]:
date_of_birth_series = filtered_df["date_of_birth"]

date_of_birth_regex = r'^\d{4}-\d{2}-\d{2}$'

matches_pattern = date_of_birth_series.str.match(date_of_birth_regex)

match_count = matches_pattern.sum()

match_count

np.int64(15257)

In [513]:
non_matching_dates = date_of_birth_series[~matches_pattern]
non_matching_dates

360       1968 October 16
697            1971/10/23
752            KBTI7FI7Y3
1045           OFH8YGZJWN
1629      January 1951 27
1994     November 1958 11
2991           PQPEUO937L
3063      1946 October 18
3533           7KGJ3C5TSW
3610           1974/06/06
3794           2000/01/06
4201     1979 February 01
4588           2003/09/21
5302           RQTF5XSXP4
5345         June 1943 28
5418     November 1963 06
5527           1998/08/14
6105     February 2005 05
6218         July 1966 08
6418           D2OZZHWOLK
7166           2001/07/28
7257      1948 October 24
8118     December 1946 09
8274           1942/05/29
8387           QTVEU5TR8H
8525           2001/10/23
9015           L3E8OV4UAC
9304           1944/11/30
9935      2005 January 27
10212          TLSTUEIKI0
10245        July 1961 14
10360          YTC82GP4XE
11204        July 1939 16
11367          O1LIA1MT1N
12178          RQI3KQXFBQ
13048     1951 January 14
13115          1IA43NTJFB
13164          1944/10/15
14103       

In [514]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15299 entries, 0 to 15298
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15299 non-null  object
 1   last_name      15299 non-null  object
 2   date_of_birth  15299 non-null  object
 3   company        15299 non-null  object
 4   email_address  15299 non-null  object
 5   address        15299 non-null  object
 6   country        15299 non-null  object
 7   country_code   15299 non-null  object
 8   phone_number   15299 non-null  object
 9   join_date      15299 non-null  object
 10  user_uuid      15299 non-null  object
dtypes: object(11)
memory usage: 1.3+ MB


Filter all rows that contain a number in the first_name column, and drop them.

In [515]:
numbers_in_name = filtered_df[filtered_df["first_name"].str.contains(r"\d", na=False)]
filtered_df = filtered_df.drop(numbers_in_name.index)
filtered_df.reset_index(drop=True)

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...
15279,Marta,Rogge,1981-03-03,Dehmel,baererklothilde@trubin.com,Ziegertstr. 60\n93330 Stollberg,Germany,DE,(05917) 549662,2000-03-29,8a77629e-7ca1-409f-b22c-c24056bd4eb1
15280,Erna,Hoffmann,1967-10-28,Atzler Seifert AG & Co. KGaA,dunjafischer@vollbrecht.de,Henkallee 186\n33456 Sankt Goarshausen,Germany,DE,+49(0)7384 51073,2018-03-13,5f57209e-8695-4863-b9e7-084a4ba02808
15281,Konstantinos,Thanel,1954-08-05,Fritsch Ehlert GmbH,rpruschke@gotthard.com,Steffi-Rose-Platz 16\n12365 Apolda,Germany,DE,(07856) 050049,2007-07-21,6f16e0ce-9b07-4479-a151-7efdd35408aa
15282,Caroline,Fisher,1975-09-27,Coleman Ltd,wardshaun@miah.org,826 Hollie park\nKhanberg\nM9J 1GP,United Kingdom,GB,(0115) 496 0754,2016-11-26,1a202edd-20aa-4787-b3b3-622fc01a9d08


In [516]:
date_of_birth_series = filtered_df["date_of_birth"]

date_of_birth_regex = r"^\d{4}-\d{2}-\d{2}$"

matches_pattern = date_of_birth_series.str.match(date_of_birth_regex)

match_count = matches_pattern.sum()

non_matching_dates = date_of_birth_series[~matches_pattern].index
non_matching_dates

Index([  360,   697,  1629,  1994,  3063,  3610,  3794,  4201,  4588,  5345,
        5418,  5527,  6105,  6218,  7166,  7257,  8118,  8274,  8525,  9304,
        9935, 10245, 11204, 13048, 13164, 14107, 14549],
      dtype='int64')

There are 27 problematic entries - this corresponds to the number of datetimes that didn't fit the YYYY MM DD format previously. We are going to need to convert these dates before we can move on.

In [517]:
filtered_df.loc[non_matching_dates]

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
360,Margaret,Akhtar,1968 October 16,"Turner, Marshall and Clarke",tcoleman@saunders-blake.org,033 Poole gateway\nLake Sharon\nCW62 6SD,United Kingdom,GB,(0191) 496 0493,1994-11-21,90f1f4cf-aca3-4c23-bc74-cc9f8e58c5ae
697,Scott,Jones,1971/10/23,"Jordan, Brown and Evans",tom73@jones.net,44 Irene plaza\nNew Abbie\nM4H 0NH,United Kingdom,GB,+449098790834,2012-10-08,9ac79783-f8e9-4639-b891-ce6b0d65028a
1629,Gertraut,Thanel,January 1951 27,Scholl GbR,kbaehr@berger.de,Werneckering 9/5\n85998 Stendal,Germany,DE,(03961) 82313,1996-01-06,35ac4c9c-ea19-4cc9-b257-100aa61238ce
1994,Peggy,Gibson,November 1958 11,Davis Group,keithespinoza@hall-stevens.info,"06263 Huynh Unions Apt. 079\nMichaelton, GA 98923",United States,US,135-127-1916,1995-05-07,d74d1885-0db3-40ed-bdfd-0548dac12337
3063,Lioba,Dobes,1946 October 18,Heinrich Stiftung & Co. KG,hthanel@lehmann.de,Rosestr. 6\n97463 Griesbach Rottal,Germany,DE,+49(0)0334 246986,1993-07-22,a272077a-e52a-48e4-a213-c93227f34d84
3610,Luke,Glover,1974/06/06,Jones-Brown,bellterence@booth-rahman.net,Flat 5\nFiona fort\nSouth Jeffreybury\nBB1 1WN,United Kingdom,GB,0115 496 0051,1994-04-25,6ef6d2fb-d48e-4385-87a5-fcc52e813a72
3794,Amber,Stewart,2000/01/06,Bennett Group,pblake@bradley.com,233 Lesley expressway\nOwenfurt\nCO7E 5UP,United Kingdom,GB,+44(0)151 4960261,1995-03-26,4a6261ad-c028-4ec6-bc91-2220fbea4887
4201,Dietlind,Karge,1979 February 01,Schlosser Karge KG,hans-josef54@ortmann.de,Alexandros-Hiller-Allee 9\n90857 Pirmasens,Germany,DE,0649651876,1993-05-23,d391bf54-2cd6-40ff-a564-18d20fab21c7
4588,Ricky,Kemp,2003/09/21,Burgess Ltd,bhopkins@morris.co.uk,475 Lane ports\nElizabethside\nCH1W 0EP,United Kingdom,GB,0161 4960570,2005-02-09,e9bb5031-c176-44ec-8d7c-f608307a2763
5345,Jaroslaw,Zimmer,June 1943 28,Wesack Wulf AG,diedrich85@waehner.net,Friedhold-Müller-Allee 84\n57937 Jena,Germany,DE,+49(0) 881581861,2006-02-06,1f5edb02-ff25-41e0-b5ab-b0c979ebdf9e


This function uses the datetime module and strptime (string parse time) to coerce all of the strings in the column to a single datetime format. The column is now a datetime64 datatype.

The ValueError just means that the string provided doesn't fit the particular format, so we move onto the next one.

If none of the formats fit, return NaT.

This function works on both the date_of_birth and the join_date columns.

In [518]:
def process_date(date_str):
  formats = ["%Y %B %d", "%Y/%m/%d", "%B %Y %d", "%Y-%m-%d"]
  # print(date_str, "Date String")
  for format in formats:
    try:
      return datetime.strptime(date_str, format)
    except ValueError:
      continue
  return pd.NaT


filtered_df["date_of_birth"] = filtered_df["date_of_birth"].apply(process_date)
filtered_df["join_date"] = filtered_df["join_date"].apply(process_date)

Let's move on to country code

In [519]:
filtered_df["country_code"].unique()

array(['DE', 'GB', 'US', 'GGB'], dtype=object)

In [520]:
country_code_errors = filtered_df[filtered_df["country_code"] == "GGB"]
country_code_errors

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
2592,Peter,Mitchell,1942-01-11,"Jennings, Taylor and Davies",mlambert@riley.com,849 Collins camp\nSouth Francisland\nAL80 8TD,United Kingdom,GGB,0121 4960043,1995-07-15,bd3e3bc5-3a25-40dd-ac0f-3f2aaf5814df
8440,Hollie,Wilson,1967-11-28,Hughes Ltd,clane@foster.com,431 Eleanor drive\nLake Yvonne\nEC28 1HL,United Kingdom,GGB,(01632) 960949,2008-10-18,686f612a-0acb-4992-ab03-3913c2011b1e
9409,Harriet,Berry,1943-12-29,Vincent LLC,paige87@knight.info,Studio 5\nJamie village\nSouth Howard\nBB0 2QQ,United Kingdom,GGB,+44(0)29 2018971,2000-12-25,0f2b4d28-dcb9-45fe-aff3-7b9135f8d89b
11656,Allan,Ryan,1975-02-28,Herbert LLC,simonlambert@barnett.info,Studio 28\nGill drive\nLake Gregory\nRH9 5LP,United Kingdom,GGB,0121 4960266,1998-05-05,6c20ecd2-499e-4883-85ff-fd50a077cbf3
12340,Cheryl,White,1987-02-17,Ward-Smith,tony04@smith.com,58 Colin mills\nNew Alex\nS2K 5BB,United Kingdom,GGB,0141 4960403,2010-02-23,347f4f09-7f10-4e5f-b116-05f14bb88944
12620,Joseph,Wright,1951-06-10,Brooks Inc,christianlee@williams-white.com,63 O'Neill falls\nWest Dawn\nN9T 5QJ,United Kingdom,GGB,+441184960597,2008-10-19,c014fa5c-396e-433b-93c1-10ae4d69e897


`filtered_df["country_code] == "GGB` is a boolean mask. Every element in the column/series specified (country_code) is given a True or False value. True if the value is "GGB", False if not.

We can locate the rows where the boolean mask is True by passing this to `loc[]`.

`filtered_df.loc[filtered_df["country_code] == "GGB"]` returns the entire row where the condition is True.

We can specify a column too, by adding the column after the boolean mask:

`filtered_df.loc[filtered_df["country_code] == "GGB", "country_code"]`

Finally, we add what we want to reasign these values to, by adding `= "GB"` to the end

In [521]:
filtered_df.loc[filtered_df["country_code"] == "GGB", "country_code"] = "GB"


In [522]:
filtered_df["country_code"] = filtered_df["country_code"].astype(str)
filtered_df.dtypes

first_name               object
last_name                object
date_of_birth    datetime64[ns]
company                  object
email_address            object
address                  object
country                  object
country_code             object
phone_number             object
join_date        datetime64[ns]
user_uuid                object
dtype: object

Validation of the user_uuids confirms that all of them are unique, and all of them follow the exact same format.

In [523]:
uuid_series = filtered_df["user_uuid"]

uuid_regex = r"^[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{12}$"

matches_pattern = uuid_series.str.match(uuid_regex)

match_count = matches_pattern.sum()

match_count

np.int64(15284)

Let's verify that nobody's birth or join date is in the future

In [532]:
current_datetime = datetime.today()

birth_after_today_series = filtered_df["join_date"] > current_datetime

birth_after_today_series.value_counts()

join_date
False    15284
Name: count, dtype: int64

Check phone numbers

In [548]:
filtered_df.loc[filtered_df["country_code"] == "DE", "phone_number"]


0           +49(0) 047905356
13          +49(0)9775 74337
14          +49(0)0406372221
18              08346 147221
21       +49 (0) 9914 457670
                ...         
15292           04846 931399
15293       +49(0)6966 27355
15294         (05917) 549662
15295       +49(0)7384 51073
15296         (07856) 050049
Name: phone_number, Length: 4708, dtype: object